In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from matplotlib.pyplot import *
# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score, average_precision_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA


import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from  matplotlib.ticker import PercentFormatter
style.use('ggplot')


file_path = 'Consumer_Complaints.csv'
consumer_data = pd.read_csv(file_path,error_bad_lines=False, index_col=False, dtype='unicode')
#consumer_data=consumer_data.dropna()
consumer_data.Product=consumer_data.Product.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data.Product=consumer_data.Product.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data.Issue=consumer_data.Issue.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

consumer_data.columns =consumer_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('?', '')
consumer_data.sub_issue=consumer_data.sub_issue.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data.sub_product=consumer_data.sub_product.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data = consumer_data.drop_duplicates(consumer_data.columns, keep='last')
consumer_data['date_received'] =[datetime.strptime(x, '%m/%d/%Y') for x in consumer_data['date_received'] ]
consumer_data['date_sent_to_company'] =[datetime.strptime(x, '%m/%d/%Y') for x in consumer_data['date_sent_to_company'] ]


dispute_data=consumer_data[(consumer_data['consumer_disputed']=='Yes')|(consumer_data['consumer_disputed']=='No')]
#dispute_data.describe()

In [2]:
####count companies disputes and create dictionary
company_counts=dispute_data['company'].value_counts()
dfcompany_counts = pd.DataFrame({'counts':company_counts.values,'dic_company':company_counts.index,'companies':company_counts.index,})

dfcompany_counts['dic_company'] =["unique" if x==1 else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]
dfcompany_counts['dic_company'] =["petite" if (x>1 and x<100) else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]
dfcompany_counts['dic_company'] =['small' if (x>100 and x<1000)  else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]
dfcompany_counts['dic_company'] =['medium' if (x>=1000 and x<5000) else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]

company_dict = dict(zip(dfcompany_counts.companies, dfcompany_counts.dic_company))
 
dispute_data['company_code']=dispute_data['company'].map(company_dict)

#sns.catplot(y="company_code", hue="consumer_disputed", kind="count",data=dispute_data)


In [3]:
dispute_data=pd.get_dummies(data=dispute_data, columns=['product'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['issue'])
#dispute_data=pd.get_dummies(data=dispute_data, columns=['sub_product'])
#dispute_data=pd.get_dummies(data=dispute_data, columns=['sub_issue'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['company_public_response'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['company_code'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['consumer_consent_provided'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['submitted_via'])

dispute_data=pd.get_dummies(data=dispute_data, columns=['company_response_to_consumer'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['timely_response'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['consumer_disputed'])


In [4]:
#dispute_data.columns.tolist()

In [5]:
features=['product_bank_account_or_service',  'product_checking_or_savings_account',  'product_consumer_loan',  'product_credit_card',  'product_credit_reporting',  'product_debt_collection',  'product_money_transfers',  'product_mortgage',  'product_other_financial_service',  'product_payday_loan',  'product_prepaid_card',  'product_student_loan',  'product_virtual_currency',  'issue_account_opening,_closing,_or_management',  'issue_account_terms_and_changes',  'issue_adding_money',  'issue_advertising,_marketing_or_disclosures',  'issue_advertising_and_marketing',  'issue_application,_originator,_mortgage_broker',  'issue_application_processing_delay',  'issue_applied_for_loan/did_not_receive_money',  'issue_apr_or_interest_rate',  'issue_arbitration',  'issue_balance_transfer',  'issue_balance_transfer_fee',  'issue_bankruptcy',  'issue_billing_disputes',  'issue_billing_statement',  "issue_can't_contact_lender",  "issue_can't_repay_my_loan",  "issue_can't_stop_charges_to_bank_account",  'issue_cash_advance',  'issue_cash_advance_fee',  'issue_charged_bank_acct_wrong_day_or_amt',  "issue_charged_fees_or_interest_i_didn't_expect",  'issue_closing/cancelling_account',  'issue_collection_debt_dispute',  'issue_collection_practices',  'issue_communication_tactics',  "issue_cont'd_attempts_collect_debt_not_owed",  'issue_convenience_checks',  'issue_credit_card_protection_/_debt_protection',  'issue_credit_decision_/_underwriting',  'issue_credit_determination',  'issue_credit_line_increase/decrease',  'issue_credit_monitoring_or_identity_protection',  'issue_credit_reporting',  "issue_credit_reporting_company's_investigation",  'issue_customer_service/customer_relations',  'issue_customer_service_/_customer_relations',  'issue_dealing_with_my_lender_or_servicer',  'issue_delinquent_account',  'issue_deposits_and_withdrawals',  'issue_disclosure_verification_of_debt',  'issue_disclosures',  'issue_excessive_fees',  'issue_false_statements_or_representation',  'issue_fees',  'issue_forbearance_/_workout_plans',  'issue_fraud_or_scam',  'issue_getting_a_loan',  'issue_identity_theft_/_fraud_/_embezzlement',  'issue_improper_contact_or_sharing_of_info',  'issue_improper_use_of_my_credit_report',  'issue_incorrect/missing_disclosures_or_info',  'issue_incorrect_exchange_rate',  'issue_incorrect_information_on_credit_report',  'issue_late_fee',  'issue_lender_damaged_or_destroyed_property',  'issue_lender_damaged_or_destroyed_vehicle',  'issue_lender_repossessed_or_sold_the_vehicle',  'issue_lender_sold_the_property',  'issue_loan_modification,collection,foreclosure',  'issue_loan_servicing,_payments,_escrow_account',  'issue_lost_or_stolen_check',  'issue_lost_or_stolen_money_order',  'issue_making/receiving_payments,_sending_money',  'issue_managing,_opening,_or_closing_account',  'issue_managing_an_account',  'issue_managing_the_line_of_credit',  'issue_managing_the_loan_or_lease',  'issue_money_was_not_available_when_promised',  'issue_opening_an_account',  'issue_other',  'issue_other_fee',  'issue_other_service_issues',  'issue_other_transaction_issues',  'issue_overdraft,_savings_or_rewards_features',  'issue_overlimit_fee',  'issue_payment_to_acct_not_credited',  'issue_payoff_process',  'issue_privacy',  'issue_problems_caused_by_my_funds_being_low',  'issue_problems_when_you_are_unable_to_pay',  "issue_received_a_loan_i_didn't_apply_for",  'issue_repaying_your_loan',  'issue_rewards',  'issue_sale_of_account',  'issue_settlement_process_and_costs',  'issue_shopping_for_a_line_of_credit',  'issue_shopping_for_a_loan_or_lease',  'issue_struggling_to_pay_mortgage',  'issue_taking/threatening_an_illegal_action',  'issue_taking_out_the_loan_or_lease',  'issue_transaction_issue',  'issue_trouble_during_payment_process',  'issue_unable_to_get_credit_report/credit_score',  'issue_unauthorized_transactions/trans._issues',  'issue_unexpected/other_fees',  'issue_unsolicited_issuance_of_credit_card',  'issue_using_a_debit_or_atm_card',  'issue_wrong_amount_charged_or_received',  'company_public_response_Company believes complaint caused principally by actions of third party outside the control or direction of the company',  'company_public_response_Company believes complaint is the result of an isolated error',  'company_public_response_Company believes complaint relates to a discontinued policy or procedure',  'company_public_response_Company believes complaint represents an opportunity for improvement to better serve consumers',  'company_public_response_Company believes it acted appropriately as authorized by contract or law',  'company_public_response_Company believes the complaint is the result of a misunderstanding',  "company_public_response_Company can't verify or dispute the facts in the complaint",  'company_public_response_Company chooses not to provide a public response',  'company_public_response_Company disputes the facts presented in the complaint',  'company_public_response_Company has responded to the consumer and the CFPB and chooses not to provide a public response',  'company_code_AMERICAN EXPRESS COMPANY',  'company_code_Alliant Capital Management LLC',  'company_code_BANK OF AMERICA, NATIONAL ASSOCIATION',  'company_code_Blatt, Hasenmiller, Leibsker & Moore, LLC',  'company_code_CAPITAL ONE FINANCIAL CORPORATION',  'company_code_CITIBANK, N.A.',  'company_code_DISCOVER BANK',  'company_code_Ditech Financial LLC',  'company_code_ENCORE CAPITAL GROUP INC.',  'company_code_EQUIFAX, INC.',  'company_code_Experian Information Solutions Inc.',  'company_code_GOLDMAN SACHS BANK USA',  'company_code_HSBC NORTH AMERICA HOLDINGS INC.',  'company_code_JPMORGAN CHASE & CO.',  'company_code_LJ Ross Associates',  'company_code_NATIONSTAR MORTGAGE',  'company_code_Navient Solutions, LLC.',  'company_code_OCWEN LOAN SERVICING LLC',  'company_code_PNC Bank N.A.',  'company_code_PORTFOLIO RECOVERY ASSOCIATES INC',  'company_code_SELECT PORTFOLIO SERVICING, INC.',  'company_code_SUNTRUST BANKS, INC.',  'company_code_SYNCHRONY FINANCIAL',  'company_code_TD BANK US HOLDING COMPANY',  'company_code_TRANSUNION INTERMEDIATE HOLDINGS, INC.',  'company_code_U.S. BANCORP',  'company_code_WELLS FARGO & COMPANY',  'company_code_medium',  'company_code_petite',  'company_code_small',  'company_code_unique',  'consumer_consent_provided_Consent not provided',  'consumer_consent_provided_Consent provided',  'consumer_consent_provided_Consent withdrawn',  'consumer_consent_provided_Other',  'submitted_via_Email',  'submitted_via_Fax',  'submitted_via_Phone',  'submitted_via_Postal mail',  'submitted_via_Referral',  'submitted_via_Web',  'company_response_to_consumer_Closed',  'company_response_to_consumer_Closed with explanation',  'company_response_to_consumer_Closed with monetary relief',  'company_response_to_consumer_Closed with non-monetary relief',  'company_response_to_consumer_Closed with relief',  'company_response_to_consumer_Closed without relief',  'company_response_to_consumer_Untimely response',  'timely_response_Yes',]
y=dispute_data[ 'consumer_disputed_Yes']
#print(len(features),len(dispute_data.columns.tolist()))
X=dispute_data[features]
train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.4, random_state = 0)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
# Create first pipeline for base without reducing features.

pipe = Pipeline([('classifier' ,LogisticRegression(solver='liblinear',class_weight={0:.29, 1:.71}))])
 

In [6]:
param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 5),
    'classifier__solver' : ['liblinear'],
    'classifier__class_weight':['balanced',{0: 0.4, 1: 0.6},{0: 0.31, 1: 0.69}]}
]

# Create grid search object

clf_log = GridSearchCV(pipe, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

# Fit on data

best_clf_log = clf_log.fit(train_X, train_y)

print(clf_log.best_params_)
y_true, y_pred_log = val_y, clf_log.predict(val_X)
print(classification_report(y_true, y_pred_log))
print(clf_log.score(val_X, val_y))


Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 28.1min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 48.0min finished


{'classifier__class_weight': 'balanced', 'classifier': LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False), 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear', 'classifier__C': 0.0001}
              precision    recall  f1-score   support

           0       0.86      0.27      0.41    247936
           1       0.21      0.83      0.34     59471

   micro avg       0.37      0.37      0.37    307407
   macro avg       0.54      0.55      0.37    307407
weighted avg       0.74      0.37      0.39    307407

0.373859411139


In [8]:
param_grid = [
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,20)),
    'classifier__max_features' : list(range(65,105,10)),
    'classifier__class_weight':['balanced',{0: 0.4, 1: 0.6},{0: 0.31, 1: 0.69}]}
]

# Create grid search object

clf_for = GridSearchCV(pipe, param_grid = param_grid, cv = 2, verbose=True, n_jobs=-1)

# Fit on data

best_clf_for= clf_for.fit(train_X, train_y)

print(clf_for.best_params_)
y_true, y_pred_for  = val_y, clf_for.predict(val_X)
print(classification_report(y_true, y_pred_for))
print(clf_for.score(val_X, val_y))

Fitting 2 folds for each of 60 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 50.2min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 142.2min finished


{'classifier': RandomForestClassifier(bootstrap=True, class_weight={0: 0.4, 1: 0.6},
            criterion='gini', max_depth=None, max_features=95,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=90, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False), 'classifier__max_features': 95, 'classifier__class_weight': {0: 0.4, 1: 0.6}, 'classifier__n_estimators': 90}
              precision    recall  f1-score   support

           0       0.81      0.98      0.88    247936
           1       0.29      0.04      0.07     59471

   micro avg       0.79      0.79      0.79    307407
   macro avg       0.55      0.51      0.48    307407
weighted avg       0.71      0.79      0.73    307407

0.794994909029


In [10]:
param_grid = [
    {'classifier' : [DecisionTreeClassifier()],
    'classifier__max_depth' : list(range(1,121,5)),
    'classifier__class_weight':['balanced',{0: 0.4, 1: 0.6},{0: 0.35, 1: 0.55},{0: 0.31, 1: 0.69}]}
]

# Create grid search object

clf_tree = GridSearchCV(pipe, param_grid = param_grid, cv = 2, verbose=True, n_jobs=-1)

# Fit on data

best_clf_tree = clf_tree.fit(train_X, train_y)

print(clf_tree.best_params_)
y_true, y_pred_tree = val_y, clf_tree.predict(val_X)
print(classification_report(y_true, y_pred_tree))
print(clf_tree.score(val_X, val_y))

Fitting 2 folds for each of 96 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed: 12.8min finished


{'classifier': DecisionTreeClassifier(class_weight={0: 0.4, 1: 0.6}, criterion='gini',
            max_depth=1, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'classifier__class_weight': {0: 0.4, 1: 0.6}, 'classifier__max_depth': 1}
              precision    recall  f1-score   support

           0       0.81      1.00      0.89    247936
           1       0.00      0.00      0.00     59471

   micro avg       0.81      0.81      0.81    307407
   macro avg       0.40      0.50      0.45    307407
weighted avg       0.65      0.81      0.72    307407

0.806539864089


In [12]:
from sklearn.linear_model import SGDClassifier
param_grid = [
    {'classifier' : [SGDClassifier()],
    'classifier__loss' : ["hinge","log"],
    'classifier__penalty':['l1','l2','elasticnet'],
    'classifier__max_iter':[5,10],
     'classifier__class_weight':['balanced',{0: 0.4, 1: 0.6},{0: 0.35, 1: 0.55},{0: 0.31, 1: 0.69}]}     
]

    
SDcl = GridSearchCV(pipe, param_grid = param_grid, cv = 2, verbose=True, n_jobs=-1)

# Fit on data

best_SDcl  = SDcl.fit(train_X, train_y)

print(SDcl.best_params_)
y_true, y_pred_SDcl = val_y, SDcl.predict(val_X)
print(classification_report(y_true, y_pred_SDcl))
print(SDcl.score(val_X, val_y))


Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  2.0min finished


{'classifier__max_iter': 5, 'classifier__penalty': 'elasticnet', 'classifier__loss': 'log', 'classifier': SGDClassifier(alpha=0.0001, average=False, class_weight={0: 0.4, 1: 0.6},
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False), 'classifier__class_weight': {0: 0.4, 1: 0.6}}
              precision    recall  f1-score   support

           0       0.81      1.00      0.89    247936
           1       0.42      0.00      0.00     59471

   micro avg       0.81      0.81      0.81    307407
   macro avg       0.61      0.50      0.45    307407
weighted avg       0.73      0.81      0.72    307407

0.806533358056


In [ ]:
from sklearn.svm import SVC

#SVC(C=1.0, kernel=’rbf’, gamma=’auto_deprecated’, class_weight=None)

param_grid = [
    {'classifier' : [SVC()],
    'classifier__kernel' : ["rbf"],#,"sigmoid",'poly'],    'classifier__gamma':["auto_deprecated","scale"],
    'classifier__class_weight':['balanced',{0: 0.3, 1: 0.7}]}     
]

    
svccl = GridSearchCV(pipe, param_grid = param_grid, cv =2 , verbose=True)#, n_jobs=-1)

# Fit on data

best_svccl  = svccl.fit(train_X, train_y)
print(classification_report(y_true, y_pred_svccl))
print(svccl.score(val_X, val_y))


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [19]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = X.corr().abs()
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)),to_drop)

There are 0 columns to remove. []
